In [ ]:
# hide
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [ ]:
import httpx
import pandas as pd

from urllib.parse import urljoin

from will_it_saturate import servers
from will_it_saturate import clients
from will_it_saturate.epochs import Epoch
from will_it_saturate.hosts import Host, HostDetails

from will_it_saturate.results import Result
from will_it_saturate.files import BenchmarkFile
from will_it_saturate.control.client import ControlClient
from will_it_saturate.repositories import SqliteRepository, register_default_tables

<IPython.core.display.Javascript object>

# Create Benchmark Config

In [ ]:
# dont_test

repository = SqliteRepository.build_repository("results.db")
register_default_tables(repository)
results_table = repository.tables["result"]

control_server_port, server_port = 8100, 5100
server_host_name = "192.168.178.113"
# server_host_name = "127.0.0.1"
server_control_host = Host(name=server_host_name, port=control_server_port)
server_control_client = ControlClient(host=server_control_host)
server_details = server_control_client.get_host_details()

# client_control_host = Host(name="127.0.0.1", port=control_server_port)
# client_control_client = ControlClient(host=client_control_host)
# client_details = client_control_client.get_host_details()

# servers_to_test = [
#     servers.FastAPIUvicornServer(host=server_host_name, port=server_port),
#     servers.DjangoGunicornWSGIServer(host=server_host_name, port=server_port),
#     servers.NginxDockerServer(host=server_host_name, port=server_port),
# ]
# clients_to_test = [
#     clients.HttpxClient(name="httpx"),
#     clients.AioHttpClient(name="aiohttp"),
#     clients.WrkClient(name="wrk", host=server_host_name, port=server_port),
# ]
# epochs_to_test = [
#     Epoch(file_size=10 ** 7, duration=10),
# ]

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

server = servers.FastAPIUvicornServer(host=server_host_name, port=server_port)

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

epoch = Epoch(file_size=10 ** 7, duration=10)
epoch.files = server_control_client.get_or_create_files(epoch)
epoch.urls = [server.file_to_url(file) for file in epoch.files]
for epoch_file in epoch.files:
    epoch_file.port = server.port
    epoch_file.hostname = server.host

<IPython.core.display.Javascript object>

In [ ]:
# dont_test


benchmark_client = clients.WrkClient(
    name="wrk", host=server_host_name, port=server_port
)
benchmark_client.create_lua_script(epoch)

<IPython.core.display.Javascript object>